In [15]:
from gensim.models import Word2Vec
from datasets import load_dataset
import os

In [16]:
TRAIN_PATH = os.path.join('..', 'POS_Tagging', 'data', 'train')
TEST_PATH = os.path.join('..', 'POS_Tagging', 'data', 'test')

In [9]:
dataset = load_dataset('parquet', data_files=[*[os.path.join(TRAIN_PATH, f'train_{i}.parquet') for i in range(9)],
                                    *[os.path.join(TEST_PATH, f'test_{i}.parquet') for i in range(9)]])

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['init_words', 'mod_words', 'pos_tags', 'sentence_length', 'unique_pos', 'starts_with'],
        num_rows: 8111724
    })
})

In [11]:
def get_sentences():
    
    gen = (['<UNK>' if pos == 'Other' else '<NUM>' if pos == 'Num' else word for word, pos in zip(sent['mod_words'], sent['pos_tags']) if pos != 'Punct'] for sent in dataset['train'])
    
    class MyIterable:
    
        def __iter__(self):
            yield from gen

    return MyIterable()

In [12]:
for idx, sent in enumerate(get_sentences()):
    if idx == 10:
        break
    print(sent)

['რუსეთის', 'სპეცსამსახურები', 'სამწუხაროდ', 'დღესაც', 'აქტიურად', 'მოქმედებენ', 'საქართველოში']
['ამერიკის', 'შეერთებულ', 'შტატებსა', 'და', 'საქართველოს', 'შორის', 'თავდაცვისა', 'და', 'უსაფრთხოების', 'სფეროში', 'თანამშრომლობა', 'და', 'პარტნიორობა', 'სასარგებლოა', 'როგორც', '<NUM>', 'სახელმწიფოსთვის', 'ისე', 'რეგიონისათვის']
['ზლატან', 'იბრაჰიმოვიჩი', 'ასეთი', 'არაფერი', 'მინახავს']
['შოპენი', 'პატარაობიდანვე', 'გამოირჩეოდა', 'თავისი', 'ნიჭით', 'არა', 'მარტო', 'მუსიკაში', 'არამედ', 'ლექსების', 'კითხვაში', 'და', 'კარიკატურების', 'ხატვაში']
['მისმა', 'სახელმა', 'და', 'სილამაზემაც', 'ნისლიანი', 'ალიონის', 'საზღვრებსაც', 'გააღწია']
['ახლოვდება', 'ჟამი', 'ჩვენი', 'სამშობლოს', 'საქართველოს', 'გათავისუფლებისა']
['საკონსტიტუციო', 'კომისიის', 'შემაჯამებელ', 'სხდომაზე', 'არც', '<NUM>', 'გადაწყვეტილება', 'არ', 'მიღებულა']
['ჩამოვედი', 'თუ', 'არა', 'ნალჩიკის', 'სპარტაკიდან', 'დამიკავშირდნენ', 'და', 'დაბრუნება', 'შემომთავაზეს']
['ვსადილობ', 'ახალი', 'ბოსტნეულით', 'განსაკუთრებულ', 'პატივს', 'ვცემ', 

# Model

In [13]:
w2v_model = Word2Vec(get_sentences(), min_count=20,
                     window=2,
                     vector_size=500,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=8,
                    epochs=30)

In [32]:
w2v_model.train(get_sentences(), total_examples=7000000, epochs=200)

(66017040, 96327699)

In [2]:
w2v_model = Word2Vec.load(os.path.join('models', "word2vec_v2.model"))

In [30]:
w2v_model.wv.most_similar(positive=['გოგო', 'ცოლი'], negative=['ბიჭი'])

[('ქმარი', 0.6007357239723206),
 ('ცოლიც', 0.5957913398742676),
 ('დედ-მამა', 0.5849706530570984),
 ('ქალიშვილი', 0.5841866135597229),
 ('რძალი', 0.5715733170509338),
 ('სიდედრი', 0.5705388784408569),
 ('ბებია', 0.5689602494239807),
 ('ქმარიც', 0.5687670707702637),
 ('მეუღლე', 0.565215528011322),
 ('საცოლე', 0.5651593208312988)]

In [21]:
w2v_model.wv.most_similar(positive=['ნელი', 'სწრაფად'], negative=['სწრაფი'])

[('ნელა', 0.5272687077522278),
 ('დინჯად', 0.45751291513442993),
 ('ელვისებურად', 0.42783600091934204),
 ('ტაატით', 0.41170790791511536),
 ('უსწრაფესად', 0.40102213621139526),
 ('უცნაურად', 0.400423526763916),
 ('გაუბედავად', 0.3972097933292389),
 ('რიტმულად', 0.39583662152290344),
 ('მდორედ', 0.3906417787075043),
 ('უცბად', 0.3888978362083435)]

In [31]:
w2v_model.wv.most_similar(positive=['ვარ', 'წავედი'], negative=['ვიყავი'])

[('მივდივარ', 0.585079550743103),
 ('წამოვედი', 0.5763711929321289),
 ('წამოვალ', 0.5667750835418701),
 ('წავალ', 0.5654860734939575),
 ('წავიდე', 0.5646795034408569),
 ('წავიდოდი', 0.5388880372047424),
 ('მოვდივარ', 0.5307322144508362),
 ('წავსულიყავი', 0.5274919867515564),
 ('ჩავიდე', 0.5157987475395203),
 ('წავედით', 0.5123860239982605)]

In [134]:
w2v_model.wv.most_similar(positive=['მთა', 'ხეები'], negative=['ხე'])

[('მთები', 0.5794955492019653),
 ('ნაპირები', 0.5692475438117981),
 ('ხრიოკი', 0.5466976761817932),
 ('ტყიანი', 0.5422216653823853),
 ('უდაბნო', 0.537442147731781),
 ('ივრის', 0.5341458916664124),
 ('ტყეები', 0.5319012403488159),
 ('შეფენილი', 0.5294755101203918),
 ('ჭალები', 0.5212216973304749),
 ('ველები', 0.5201641917228699)]

In [135]:
w2v_model.wv.most_similar(positive=['ქალი', 'მეფე'], negative=['კაცი'])

[('დედოფალი', 0.5266863107681274),
 ('დედოფალ', 0.4737202823162079),
 ('პრინცი', 0.4736841022968292),
 ('ქალწული', 0.4696141481399536),
 ('მეფეც', 0.46924567222595215),
 ('გურანდუხტ', 0.46605610847473145),
 ('მზეთუნახავი', 0.46064096689224243),
 ('დედოფლისა', 0.4519679844379425),
 ('ორლეანელი', 0.45065176486968994),
 ('უფლისწული', 0.44770342111587524)]

In [131]:
w2v_model.wv.most_similar('მთა')

[('უდაბნო', 0.6239278316497803),
 ('მთაც', 0.6182738542556763),
 ('მთები', 0.5892217755317688),
 ('მთაა', 0.588892936706543),
 ('კლდე', 0.5619916915893555),
 ('ნაპირები', 0.5547749996185303),
 ('ჭალა', 0.5507076382637024),
 ('ნაპირი', 0.5419477224349976),
 ('მღვიმე', 0.5412144064903259),
 ('მდელო', 0.5390500426292419)]

In [137]:
w2v_model.save(os.path.join('models', "word2vec_v2.model"))

# Compare to fastext Word2vec

In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pybind11>=2.2 from https://files.pythonhosted.org/packages/06/55/9f73c32dda93fa4f539fafa268f9504e83c489f460c380371d94296126cd/pybind11-2.11.1-py3-none-any.whl.metadata
  Using cached pybind11-2.11.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=311173 sha256=5ae78a7d8545badf8cdbc069ab52b6f6f31a728d83540f56d72976924b241169
  Stored in directory: /home/jupyter/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext.util
fasttext.util.download_model('ka', if_exists='ignore')  # English
model = fasttext.load_model('cc.ka.300.bin')

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

